<a href="https://colab.research.google.com/github/eeuunnjjiii/Franchise-Guide/blob/main/4.Modeling/4-2.Modeling_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# 한글 깨짐 방지
!apt-get install fonts-nanum*
!apt-get install fontconfig
import matplotlib.font_manager as fm
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
fm._rebuild()

# 화질 선명하게 설정
%config InlineBackend.figure_format='retina'

# DataFrame 컬럼 전체보기 설정
pd.set_option('display.max_columns', None) 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'fonts-nanum-eco' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-gothic-light' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-coding' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-extra' for glob 'fonts-nanum*'
fonts-nanum is already the newest version (20170925-1).
fonts-nanum-coding is already the newest version (2.5-1).
fonts-nanum-eco is already the newest version (1.000-6).
fonts-nanum-extra is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fontconfig is already the newest version (2.12.6-0ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
NanumGothic Eco


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 데이터 가져오기
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/franchise_data/total_eda.csv', index_col=0).reset_index(drop=True)

#1. 기준모델

In [5]:
# 타겟지정
target = '가맹점평균매출액'

#sklearn 사용
from sklearn.metrics import mean_absolute_error
y_pred=[df[target].mean()]*len(df)
mae=mean_absolute_error(df[target],y_pred)
print('기준모델 mae : ',mae) ## 이상치가 있기 때문에, mae 지표 사용

기준모델 mae :  179816.18196185297


#3. 데이터 분리

In [6]:
#Hold-out 방식으로 훈련, 검증, 테스트 데이터셋 분리 (타겟데이터 비율 맞추기)
from sklearn.model_selection import train_test_split
train,test = train_test_split(df,train_size=0.8, test_size=0.2, random_state=2)
train,val = train_test_split(train,train_size=0.8, test_size=0.2, random_state=2)

train.shape, val.shape, test.shape

((5328, 36), (1333, 36), (1666, 36))

In [7]:
#데이터 특성/타겟 분리

features=train.drop(columns=[target]).columns

X_train=train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

#4. 특성 줄이기

In [8]:
def remove_feature(df):
    
    # 정보공개서에서 쉽게 열람할 수 없는 특성 삭제
    df = df.drop(['부채비율','자기자본비율','영업이익률','매출액순이익률','자기자본순이익률','폐점','폐업률'],axis=1)

    # data leakage 특성 제거
    df = df.drop(['면적당평균매출액'],axis=1)

    # data cardinality 높은 특성 제거
    df =df.drop(['브랜드','상호'], axis=1)
    
    # 중복되는 특성 제거
    df = df.drop(['면적당인테리어비용','기준년도','가맹사업개시일'],axis=1)
    
    # 합계(가맹비+교육비+보증금+기타비용)에 해당되는 특성 제거
    df = df.drop(['가맹비','교육비','보증금','기타비용'],axis=1)

    return df

X_train=remove_feature(X_train)
X_val=remove_feature(X_val)
X_test=remove_feature(X_test)

#5. 머신러닝 적용

##1) Linear Regression 모델

In [9]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 7.6 MB/s 


In [10]:
from sklearn.pipeline import make_pipeline
from category_encoders import TargetEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

pipe1=make_pipeline(TargetEncoder(), ## target에 맞춰 범주형 인코딩
                   RobustScaler(), ## 이상치에 강함 (평균과 분산 대신 중앙값과 사분위값 사용)
                   LinearRegression()) ## 가장 기본모델

pipe1.fit(X_train,y_train)
y_pred = pipe1.predict(X_val)
print('검증 데이터 MAE(Linear):', mean_absolute_error(y_val,y_pred))
print('검증 데이터 R2(Linear):', r2_score(y_val,y_pred))

검증 데이터 MAE(Linear): 135604.39144220008
검증 데이터 R2(Linear): 0.27838029511822737


##2) RandomForests 모델

In [11]:
from sklearn.ensemble import RandomForestRegressor

pipe2=make_pipeline(TargetEncoder(), ## target에 맞춰 범주형 인코딩
                   RobustScaler(), ## 이상치에 강함 (평균과 분산 대신 중앙값과 사분위값 사용)
                   RandomForestRegressor(random_state=2)) ## 가장 기본모델

pipe2.fit(X_train,y_train)
y_pred = pipe2.predict(X_val)
print('검증 데이터 MAE(RandomForests):', mean_absolute_error(y_val,y_pred))
print('검증 데이터 R2(RandomForests):', r2_score(y_val,y_pred))

검증 데이터 MAE(RandomForests): 104855.22068267065
검증 데이터 R2(RandomForests): 0.4145206431218339


##3) XGBoost 모델

In [12]:
from xgboost import XGBRegressor

pipe_xg=make_pipeline(TargetEncoder(),
                      RobustScaler(),
                      XGBRegressor(n_jobs=-1, random_state=2))

pipe_xg.fit(X_train,y_train);
y_pred = pipe_xg.predict(X_val)
print('검증 데이터 MAE(XGBoost):', mean_absolute_error(y_val,y_pred))
print('검증 데이터 R2(XGBoost):', r2_score(y_val,y_pred))

[12:53:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
검증 데이터 MAE(XGBoost): 114894.75261393473
검증 데이터 R2(XGBoost): 0.38173445513902615


- 모든 평가지표에서 더 나은 성능을 보인 RandomForest 모델 선택

##4) RandomForest 모델 하이퍼파라미터 수정

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

pipe=make_pipeline(TargetEncoder(), ## target에 맞춰 범주형 인코딩
                   RobustScaler(), ## 이상치에 강함 (평균과 분산 대신 중앙값과 사분위값 사용)
                   RandomForestRegressor(random_state=2)) ## 가장 기본모델

# 하이퍼파라미터 범위 지정

dists={
    'targetencoder__smoothing' : [1.,10.,50.,100.,500.],
    'targetencoder__min_samples_leaf' : randint(1,50),
    'robustscaler__quantile_range':[(25.0,75.0),(10.0,90.0),(1.0,99.0)],
    'randomforestregressor__n_estimators' : randint(1,500),
    'randomforestregressor__max_depth': [5, 10, 15, 20, None],
    'randomforestregressor__max_features' : uniform(0,1)
}

# 시행
clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    cv=5, 
    scoring='r2',  
    verbose=1,
    n_jobs=-1,
    random_state=2
    )

clf.fit(X_train, y_train);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.4min finished


In [14]:
print('최적 하이퍼파라미터: ', clf.best_params_)
print('score: ', clf.best_score_)

최적 하이퍼파라미터:  {'randomforestregressor__max_depth': 20, 'randomforestregressor__max_features': 0.42036780208748903, 'randomforestregressor__n_estimators': 163, 'robustscaler__quantile_range': (10.0, 90.0), 'targetencoder__min_samples_leaf': 32, 'targetencoder__smoothing': 100.0}
score:  0.6210075764693426


In [17]:
dists1={
    'targetencoder__smoothing' : [100.,110.,120.,130.],
    'targetencoder__min_samples_leaf' : randint(30,40),
    'robustscaler__quantile_range':[(25.0,75.0),(10.0,90.0),(1.0,99.0)],
    'randomforestregressor__n_estimators' : randint(160,170),
    'randomforestregressor__max_depth': randint(15,25),
    'randomforestregressor__max_features' : uniform(0,1)
}

# 시행
clf2 = RandomizedSearchCV(
    pipe, 
    param_distributions=dists1, 
    cv=5, 
    scoring='r2',  
    verbose=1,
    n_jobs=-1,
    random_state=2
    )

clf2.fit(X_train, y_train);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.3min finished


In [18]:
print('최적 하이퍼파라미터: ', clf2.best_params_)
print('score: ', clf2.best_score_)

최적 하이퍼파라미터:  {'randomforestregressor__max_depth': 23, 'randomforestregressor__max_features': 0.35367537868439536, 'randomforestregressor__n_estimators': 169, 'robustscaler__quantile_range': (1.0, 99.0), 'targetencoder__min_samples_leaf': 35, 'targetencoder__smoothing': 120.0}
score:  0.6208260420332752


- clf 모델 사용

## 5) 최종 모델

In [28]:
# 최적 모델 생성
model=make_pipeline(TargetEncoder(min_samples_leaf=32, smoothing=100.0), 
                   RobustScaler(quantile_range=(10.0, 90.0)), 
                   RandomForestRegressor(n_estimators=163, max_features=0.42036780208748903, max_depth=20, random_state=2))
model.fit(X_train,y_train)
y_pred = model.predict(X_val)
print('검증 데이터 MAE:', mean_absolute_error(y_val,y_pred))
print('검증 데이터 R2:', r2_score(y_val,y_pred))

검증 데이터 MAE: 103325.26228779185
검증 데이터 R2: 0.43056601021454266


In [29]:
# test data 확인
y_pred = model.predict(X_test)
print('검증 데이터 MAE:', mean_absolute_error(y_test,y_pred))
print('검증 데이터 R2:', r2_score(y_test,y_pred))

검증 데이터 MAE: 98638.1432373667
검증 데이터 R2: 0.6581444467933129


#6. 모델 해석

##1)  permutation importance : 특성 중요도

In [30]:
!pip install eli5

     |████████████████████████████████| 106 kB 35.0 MB/s 


In [35]:
import eli5
from eli5.sklearn import PermutationImportance

permuter=PermutationImportance(model, scoring='r2',random_state=2)

encoder = TargetEncoder(min_samples_leaf=32, smoothing=100.0)
X_train_encoded = encoder.fit_transform(X_train,y_train)
X_test_encoded = encoder.transform(X_test)
permuter.fit(X_test_encoded,y_test);

In [37]:
# 특성별 score 확인
eli5.show_weights(
    permuter, 
    top=None, 
    feature_names=X_test_encoded.columns.tolist()
)

Weight,Feature
0.3426 ± 0.0147,합계
0.1814 ± 0.0234,기준면적
0.0445 ± 0.0075,총인테리어비용
0.0423 ± 0.0096,당기순이익
0.0355 ± 0.0092,가맹사업일수
0.0268 ± 0.0058,영업이익
0.0220 ± 0.0101,매출액
0.0190 ± 0.0062,계약해지
0.0158 ± 0.0038,자본
0.0126 ± 0.0042,부채


- 회귀 모델에서 중요한 특성은 합계, 기준면적, 총인테리어비용 순